Create the school calendars for a given year using the data from the Excel file.

In [ ]:


from schemas import SchoolNode, CalendarYearNode, TermNode, TermBreakNode, WeekNode, DayNode, PeriodNode
from schemas import SchoolHasCalendarYear, CalendarYearHasTerm, CalendarYearHasTermBreak, CalendarYearHasWeek, CalendarYearHasDay, CalendarYearHasPeriod, TermHasTermBreak, TermHasWeek, TermHasDay, TermHasPeriod, TermBreakHasWeek, TermBreakHasDay, WeekHasDay, WeekHasPeriod, DayHasPeriod, CalendarYearHasNextCalendarYear, CalendarYearHasPreviousCalendarYear, TermHasNextTerm, TermHasPreviousTerm, TermBreakHasNextTermBreak, TermBreakHasPreviousTermBreak, WeekHasNextWeek, WeekHasPreviousWeek, DayHasNextDay, DayHasPreviousDay, PeriodHasNextPeriod, PeriodHasPreviousPeriod
import schema_tools as schema_tools
import get_planner as planner
import neontology_tools as neon
import neo4j_driver_tools as neo4j

import json
import pandas as pd
from pydantic import ValidationError, BaseModel
from neontology import BaseNode, BaseRelationship
from typing import Optional, ClassVar, List # Delete after removing dummy code
from datetime import datetime, timedelta, date, time, timezone

In [ ]:

# Function to convert Excel serial date to datetime.date
def convert_excel_date(excel_date):
    logging
    if pd.isna(excel_date) or excel_date == 'Null':
        logging.pedantic(f"Excel date is NaN or Null: {excel_date}")
        return None
    if isinstance(excel_date, datetime):
        logging.pedantic(f"Excel date is already a datetime: {excel_date}")
        return excel_date.date()
    else:
        # Assuming excel_date is a fraction of the day
        logging.pedantic(f"Excel date is a fraction of the day: {excel_date}")
        return (datetime(1899, 12, 30) + timedelta(days=excel_date)).date()

# Function to convert Excel serial time to datetime.time
def convert_excel_time(excel_time):
    logging.pedantic(f"Converting Excel time: {excel_time}")
    if pd.isna(excel_time) or excel_time == 'Null':
        logging.pedantic(f"Excel time is NaN or Null: {excel_time}")
        return None
    if isinstance(excel_time, datetime):
        logging.pedantic(f"Excel time is already a datetime: {excel_time}")
        return excel_time.time()
    else:
        # Assuming excel_time is a fraction of the day
        logging.pedantic(f"Excel time is a fraction of the day: {excel_time}")
        total_seconds = int(24 * 60 * 60 * excel_time)
        return (datetime(1900, 1, 1) + timedelta(seconds=total_seconds)).time()

# Function to convert date strings to datetime.date
def convert_date_string(date_string):
    logging.pedantic(f"Converting date string: {date_string}")
    if pd.isna(date_string) or date_string == 'Null':
        logging.warning(f"Date string is NaN or Null: {date_string}")
        return None
    return pd.to_datetime(date_string).date()

# Function to convert time strings to datetime.time
def convert_time_string(time_string):
    logging.pedantic(f"Converting time string: {time_string}")
    if pd.isna(time_string) or time_string == 'Null':
        logging.warning(f"Time string is NaN or Null: {time_string}")
        return None
    return pd.to_datetime(time_string, format='%H:%M:%S').time()

In [ ]:
# Define conversion functions
def convert_to_date(date_str):
    logging.debug(f"Converting {date_str} to date object...")
    try:
        logging.pedantic(f"Converting {date_str} to date object...")
        return pd.to_datetime(date_str, dayfirst=True).date()  # Assuming day comes first in your date format
    except ValueError:
        logging.error(f"Error converting {date_str} to date object.")
        # Handle incorrect format
        return None

def convert_to_time(time_str):
    logging.debug(f"Converting {time_str} to time object...")
    try:
        logging.pedantic(f"Converting {time_str} to time object...")
        return pd.to_datetime(time_str, format='%H:%M:%S').time()
    except ValueError:
        logging.error(f"Error converting {time_str} to time object.")
        # Handle incorrect format
        return None

def convert_data_based_on_type(row):
    logging.debug(f"Converting data based on type...")
    data_type = row['DataType']
    data_value = row['Data']
    if data_type == 'Date':
        logging.pedantic(f"Converting {data_value} to date object...")
        return convert_to_date(data_value)
    elif data_type == 'Time':
        logging.pedantic(f"Converting {data_value} to time object...")
        return convert_to_time(data_value)
    elif data_type == 'Integer':
        logging.pedantic(f"Converting {data_value} to integer object...")
        return int(data_value)
    elif data_type == 'Text':
        logging.pedantic(f"Converting {data_value} to string object...")
        return str(data_value)
    else:
        logging.error(f"Invalid data type: {data_type}")
        return None

# Function to read term dates from the Excel sheet
def read_term_dates(excel_path, sheet_name='CalendarLookup'):
    logging.info(f"Reading term dates from {excel_path}...")
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    term_dates = {}
    for index, row in df.iterrows():
        logging.debug(f"Reading row {index}...")
        # Convert Identifier to string to avoid TypeError
        identifier = str(row['Identifier'])
        if 'Term' in identifier:
            term_name = identifier.replace('Start', '').replace('End', '')
            logging.debug(f"Term name: {term_name}")
            converted_data = convert_data_based_on_type(row)
            logging.debug(f"Converted data: {converted_data}")
            if 'Start' in identifier:
                logging.pedantic(f"Adding start date for {term_name}...")
                term_dates.setdefault(term_name, {})['start_date'] = converted_data
            elif 'End' in identifier:
                logging.pedantic(f"Adding end date for {term_name}...")
                term_dates[term_name]['end_date'] = converted_data
    logging.info(f"Term dates read from {excel_path}.")
    return term_dates

# Additional function to create weeks and days
def create_weeks_and_days(weeks_df, days_df, term_nodes, teaching_periods):
    logging.info(f"Creating WeekNode and DayNode instances...")
    # Create WeekNode and DayNode instances
    weeks = []
    days = []

    # Create weeks
    for index, week_data in weeks_df.iterrows():
        logging.debug(f"Creating WeekNode instance for {week_data['Identifier']}...")
        # Calculate week_end by adding 6 days to the week_start
        week_start = convert_date_string(week_data['WeekStartDate'])
        week_end = week_start + timedelta(days=6)

        week_node = WeekNode(
            week_id=week_data['Identifier'],
            week_start=week_start,
            week_end=week_end,  # This now uses the calculated week_end date
            week_type=week_data['WeekType'],
            week_notes='Created by KevlarAI'
        )
        weeks.append(week_node)
        logging.debug(f"WeekNode instance created for {week_data['Identifier']}.")

    # Create days and periods
    for index, day_data in days_df.iterrows():
        logging.debug(f"Creating DayNode instance for {day_data['Identifier']}...")
        if day_data['AcademicDay'] == 'Yes':  # Check if it's an academic day
            logging.debug(f"Creating DayNode instance for {day_data['Identifier']}...")
            day_node = DayNode(
                day_id=day_data['Identifier'],
                date=convert_date_string(day_data['Date']),
                day=day_data['Day'],
                day_modifier=day_data['DayModifier'],
                auto_agenda=day_data['DayAutoAgenda'],
                agenda_heading=day_data['DayAgendaHeading'],
                agenda_notes=day_data['DayAgendaNotes']
            )
            days.append(day_node)
            logging.debug(f"DayNode instance created for {day_data['Identifier']}.")

            # Create periods for each academic day
            for period_num in range(1, teaching_periods + 1):
                logging.debug(f"Creating PeriodNode instance for {day_data['Identifier']}_P{period_num}...")
                period_node = PeriodNode(
                    period_id=f"{day_data['Identifier']}_P{period_num}",
                    period_name=f"Period {period_num}",
                    # Example: period_start_time and period_end_time need to be retrieved from the data
                    period_start_time=..., 
                    period_end_time=...,
                    period_is_type='Academic',
                    period_mod='',
                    period_notes=f"Period {period_num} on {day_data['Date']}"
                )
                logging.debug(f"PeriodNode instance created for {day_data['Identifier']}_P{period_num}.")
                # Add relationship creation here if necessary

    # Return the created nodes
    return weeks, days


# Function to create TermNode instances
def create_terms_from_dates(term_dates, calendar_year_id):
    logging.info(f"Creating TermNode instances from term dates...")
    terms = []
    for term_name, dates in term_dates.items():
        logging.debug(f"Creating TermNode instance for {term_name}...")
        term_node = TermNode(
            term_id=term_name,
            term_start_date=dates['start_date'],
            term_end_date=dates['end_date'],
            term_notes=f'Created by KevlarAI for calendar year {calendar_year_id}'
        )
        terms.append(term_node)
    logging.info(f"TermNode instances created for {len(terms)} terms.")
    return terms

# Function to create relationships between calendar year and terms
def create_calendar_year_term_relationships_old(driver, calendar_year, terms):
    logging.info(f"Creating relationships between calendar year and terms...")
    for term in terms:
        logging.debug(f"Creating relationship between {calendar_year.calendar_year_id} and {term.term_id}...")
        calendar_year_has_term = CalendarYearHasTerm(source=calendar_year, target=term)
        logging.debug(f"Relationship created: {calendar_year_has_term}")
        neon.create_or_merge_neontology_relationship(driver, calendar_year_has_term, operation='merge')
        logging.info(f"Relationship created between {calendar_year.calendar_year_id} and {term.term_id}")

In [ ]:
neon.init_neo4j_connection()
neo4j_driver = neo4j.get_neo4j_driver()


In [ ]:
# Create SchoolNodes from a list
schools = [
    #SchoolNode(school_id='KCAR', school_name='King Charles School', school_org_type='Academy', school_address='King Charles School, Holly Hall Road, Dudley, West Midlands, DY2 0TZ', school_website='https://www.kingcharlesschool.co.uk/'),
    #SchoolNode(school_id='KEV', school_name='Kevlar Academy', school_org_type='Academy', school_address='Kevlar Academy, 123 Fake Street, London, SW1A 1AA', school_website='https://www.kevlaracademy.com/')
]
# Now create a routine to create schools from a list
for school in schools:
    neon.create_or_merge_neontology_node(school, operation='merge')
    logging.info(f"School created: {school}")


In [ ]:
# Read the excel file and create a dataframe
planner = planner.get_excel_sheets('planner.xlsx ')


In [ ]:
# Read the sheet dataframes from the Excel file
calendar_lookup_df = planner['calendarlookup_df']
weeks_lookup_df = planner['weekslookup_df']
days_lookup_df = planner['dayslookup_df']

# Display the first few rows of each dataframe for review
calendar_lookup_df.head(), weeks_lookup_df.head(), days_lookup_df.head()

# Scan all dataframes for NaN values and replace them with default values
default_values = {
    'Identifier': 'Null',
    'Data': 'Null',
    'DataType': 'Null',
    'Description': 'Null',
    'AcademicDay': 'Null',
    'Date': 'Null',
    'Day': 'Null',
    'DayModifier': 'Null',
    'DayAutoAgenda': 'Null',
    'DayAgendaHeading': 'Null',
    'DayAgendaNotes': 'Null',
    'AcademicWeek': 'Null',
    'WeekStartDate': 'Null',
    'WeekType': 'Null',
    'WeekAgendaHeading': 'Null',
    'WeekAgendaNotes': 'Null',
    'CalendarYearNotes': 'Null',
}
#logging.pedantic(f"Replacing NaN values with default values: {default_values}")
#calendar_lookup_df = calendar_lookup_df.apply(lambda x: replace_nan_with_default(x, default_values), axis=1)
#weeks_lookup_df = weeks_lookup_df.apply(lambda x: replace_nan_with_default(x, default_values), axis=1)
#days_lookup_df = days_lookup_df.apply(lambda x: replace_nan_with_default(x, default_values), axis=1)

# Convert the Excel dates to datetime.date
logging.pedantic(f"Converting Excel dates to datetime.date")
days_lookup_df['Date'] = days_lookup_df['Date'].apply(convert_date_string)
weeks_lookup_df['WeekStartDate'] = weeks_lookup_df['WeekStartDate'].apply(convert_date_string)
# There are specific rows in the calendar lookup that need to be converted to datetime.date
# We will find them by using a list of indentifiers
date_identifiers = ['FirstStaffDay', 'LastStaffDay', 'Term1Start', 'Term1End', 'Term2Start', 'Term2End', 'Term3Start', 'Term3End', 'Term4Start', 'Term4End', 'Term5Start', 'Term5End', 'Term6Start', 'Term6End']
time_identifiers = ['P.Reg.PeriodStartTime', 'P.Reg.PeriodEndTime', 'P.1.PeriodStartTime', 'P.1.PeriodEndTime', 'P.2.PeriodStartTime', 'P.2.PeriodEndTime', 'P.3.PeriodStartTime', 'P.3.PeriodEndTime', 'P.4.PeriodStartTime', 'P.4.PeriodEndTime', 'P.5.PeriodStartTime', 'P.5.PeriodEndTime', 'P.6.PeriodStartTime', 'P.6.PeriodEndTime', 'P.Brk.PeriodStartTime', 'P.Brk.PeriodEndTime', 'P.Lun.PeriodStartTime', 'P.Lun.PeriodEndTime']

# Convert the Excel calendar lookup dates to datetime.date
logging.pedantic(f"Converting Excel calendar lookup dates to datetime.date")
calendar_lookup_df.loc[calendar_lookup_df['Identifier'].isin(date_identifiers), 'Data'] = calendar_lookup_df.loc[calendar_lookup_df['Identifier'].isin(date_identifiers), 'Data'].apply(convert_date_string)

# Convert the Excel times to datetime.time
logging.pedantic(f"Converting Excel times to datetime.time")
calendar_lookup_df.loc[calendar_lookup_df['Identifier'].isin(time_identifiers), 'Data'] = calendar_lookup_df.loc[calendar_lookup_df['Identifier'].isin(time_identifiers), 'Data'].apply(convert_time_string)

calendar_lookup_json = calendar_lookup_df.to_json(orient='records')
days_lookup_json = days_lookup_df.to_json(orient='records')
weeks_lookup_json = weeks_lookup_df.to_json(orient='records')

# Combine the JSON objects into a single JSON structure
data = {
    "CalendarLookup": calendar_lookup_json,
    "DaysLookup": days_lookup_json,
    "WeeksLookup": weeks_lookup_json
}

# Parse the individual sheets
calendar_data = json.loads(data['CalendarLookup'])
logging.pedantic(f"Calendar data: {calendar_data}")
days_data = json.loads(data['DaysLookup'])
logging.pedantic(f"Days data: {days_data}")
weeks_data = json.loads(data['WeeksLookup'])
logging.pedantic(f"Weeks data: {weeks_data}")

In [ ]:
# Create the calendar year from the calendar_lookup_df
calendar_year = CalendarYearNode(
    calendar_year_id='2023-2024',
    calendar_year_start_date=calendar_data[3]['Data'],
    calendar_year_end_date=calendar_data[14]['Data'],
    calendar_year_notes='Created by KevlarAI'
)
logging.info(f"Calendar year created: {calendar_year}")
# Merge the calendar year node
neon.create_or_merge_neontology_node(calendar_year, operation='merge')

In [ ]:
# Create a school calendar year node for each school from the calendar data frame
for school in schools:
    logging.info(f"Finding SchoolNode instance for {school.school_id}...")
    school_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'School', {'school_id': school.school_id})
    if not school_nodes:  # Check if the list is empty
        logging.error(f"No school node found for ID {school.school_id}")
        continue  # Skip this iteration if no node is found
    for school_node_object in school_nodes:
        school_node_properties = dict(school_node_object._properties)  # Access the node properties
        logging.info(f"SchoolNode properties: {school_node_properties}")
        school_node = SchoolNode(**school_node_properties)
        logging.info(f"SchoolNode instance created for {school.school_id}: {school_node}")
        # Create the relationship between the school and the calendar year
        logging.info(f"Creating relationship between {school.school_id} and {calendar_year.calendar_year_id}...")
        school_has_calendar_year = SchoolHasCalendarYear(source=school_node, target=calendar_year)
        neon.create_or_merge_neontology_relationship(school_has_calendar_year, operation='merge')
        logging.info(f"Relationship created between {school.school_id} and {calendar_year.calendar_year_id}")

In [ ]:
# Get the calendar year node from the database using neo4j_driver
logging.info(f"Finding CalendarYearNode instance for {calendar_year.calendar_year_id}...")
calendar_year_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'CalendarYear', {'calendar_year_id': calendar_year.calendar_year_id})
if not calendar_year_nodes:  # Check if the list is empty
    logging.error(f"No calendar year node found for ID {calendar_year.calendar_year_id}")
    sys.exit(1)  # Exit the script if no node is found
# Read the start and end dates from the calendar year node
logging.info(f"Reading start and end dates from {calendar_year.calendar_year_id}...")
calendar_year_node_object = calendar_year_nodes[0]  # Get the first node from the list
calendar_year_node_properties = dict(calendar_year_node_object._properties)  # Access the node properties
logging.info(f"CalendarYearNode properties: {calendar_year_node_properties}")
calendar_year_start_date_neo4j = calendar_year_node_properties['calendar_year_start_date']
calendar_year_end_date_neo4j = calendar_year_node_properties['calendar_year_end_date']

# Convert Neo4j date to Python datetime.date
calendar_year_start_date = date(calendar_year_start_date_neo4j.year, calendar_year_start_date_neo4j.month, calendar_year_start_date_neo4j.day)
calendar_year_end_date = date(calendar_year_end_date_neo4j.year, calendar_year_end_date_neo4j.month, calendar_year_end_date_neo4j.day)

logging.info(f"Calendar year start date: {calendar_year_start_date}")
logging.info(f"Calendar year end date: {calendar_year_end_date}")

In [ ]:
# Create a DayNode instance for each day between the start and end dates
# Use the excel data to add information to the DayNode instances
logging.info(f"Creating DayNode instances for {calendar_year.calendar_year_id}...")
days = []
n = 0
for single_date in pd.date_range(calendar_year_start_date, calendar_year_end_date):
    # Set day_modifier to 'Weekend' if the day is a weekend
    # Find the corresponding day data in days_data
    day_data = next((item for item in days_data if item['Date'] == single_date.strftime('%Y-%m-%d')), None)
    # Set day_modifier based on day_data or default to 'School day'
    if single_date.weekday() in [5, 6]:
        day_modifier = 'Weekend'
    elif day_data and day_data['DayModifier'] == 'H':
        day_modifier = 'Holiday'
    elif day_data and day_data['DayModifier'] == 'C':
        day_modifier = 'Cancelled'
    else:
        day_modifier = 'School day'
    logging.debug(f"Creating DayNode instance for {single_date}...")
    day_node = DayNode(
        day_id=n,
        date=single_date.strftime('%Y-%m-%d'),
        day=single_date.strftime('%A'),
        day_modifier=day_modifier,
        auto_agenda='',
        agenda_heading='',
        agenda_notes=''
    )
    days.append(day_node)
    logging.debug(f"DayNode instance created for {single_date}")
    # Merge the DayNode instance
    neon.create_or_merge_neontology_node(day_node, operation='merge')
    # Create the relationship between the calendar year and the day
    logging.debug(f"Creating relationship between {calendar_year.calendar_year_id} and {single_date}...")
    calendar_year_has_day = CalendarYearHasDay(source=calendar_year, target=day_node)
    neon.create_or_merge_neontology_relationship(calendar_year_has_day, operation='merge')
    logging.debug(f"Relationship created between {calendar_year.calendar_year_id} and {single_date}")
    n += 1
logging.info(f"{n} DayNode instances created for {calendar_year.calendar_year_id}.")

In [ ]:
# Identify unique weeks and create WeekNode instances
logging.info(f"Creating WeekNode instances for {calendar_year.calendar_year_id}...")
weeks = []
week_identifiers = set()  # To keep track of unique week identifiers
n = 0
for single_date in pd.date_range(calendar_year_start_date, calendar_year_end_date):
    # Determine the start of the week (Monday)
    week_start = single_date - timedelta(days=single_date.weekday())
    week_end = week_start + timedelta(days=6)
    week_id = week_start.strftime('%Y-%m-%d')

    if week_id not in week_identifiers:
        week_identifiers.add(week_id)
        week_data = weeks_lookup_df.loc[weeks_lookup_df['WeekStartDate'] == week_start.strftime('%Y-%m-%d')]

        # Determine the WeekType
        if not week_data.empty and week_data['WeekType'].values[0] in ['A', 'B']:
            week_type = week_data['WeekType'].values[0]
        else:
            week_type = ''  # Default to none if the type is not 'A' or 'B' or data is empty

        logging.debug(f"Creating WeekNode instance for week starting {week_start}...")
        week_node = WeekNode(
            week_id=n,
            week_start=week_start.strftime('%Y-%m-%d'),
            week_end=week_end.strftime('%Y-%m-%d'),
            week_type=week_type,
            week_notes='Created by KevlarAI'
        )
        weeks.append(week_node)
        # Merge the WeekNode instance and create relationships...
        neon.create_or_merge_neontology_node(week_node, operation='merge')
        # Create the relationship between the calendar year and the week
        logging.debug(f"Creating relationship between {calendar_year.calendar_year_id} and {week_start}...")
        calendar_year_has_week = CalendarYearHasWeek(source=calendar_year, target=week_node)
        neon.create_or_merge_neontology_relationship(calendar_year_has_week, operation='merge')
        logging.debug(f"Relationship created between {calendar_year.calendar_year_id} and {week_start}")
        n += 1

In [ ]:
# Merge a relationship between the weeks and days
logging.info("Merging relationships between weeks and days...")
for week_node in weeks:
    week_start_date = week_node.week_start  # Assuming week_start is already a datetime.date object
    week_end_date = week_node.week_end  # Assuming week_end is already a datetime.date object

    logging.debug(f"Week start date: {week_start_date}")
    logging.debug(f"Week end date: {week_end_date}")
    week_days = [day for day in days if week_start_date <= day.date <= week_end_date]

    for day_node in week_days:
        logging.debug(f"Creating relationship between {week_node.week_id} and {day_node.date}...")
        week_has_day = WeekHasDay(source=week_node, target=day_node)
        neon.create_or_merge_neontology_relationship(week_has_day, operation='merge')
        logging.debug(f"Relationship created between {week_node.week_id} and {day_node.date}")


In [ ]:
# Read the term dates from the Excel file
excel_path = 'planner.xlsx'  # Replace with your Excel file path
term_dates = read_term_dates(excel_path)
logging.info(f"Term dates: {term_dates}")


In [ ]:
# Create the term nodes from the term dates
term_nodes = create_terms_from_dates(term_dates, calendar_year.calendar_year_id)
logging.info(f"Term nodes: {term_nodes}")
# Merge the term nodes
for term_node in term_nodes:
    neon.create_or_merge_neontology_node(term_node, operation='merge')
    logging.info(f"Term node created: {term_node}")

# Create the relationships between the calendar year and the terms
for term_node in term_nodes:
    logging.info(f"Creating relationship between {calendar_year.calendar_year_id} and {term_node.term_id}...")
    calendar_year_has_term = CalendarYearHasTerm(source=calendar_year, target=term_node)
    neon.create_or_merge_neontology_relationship(calendar_year_has_term, operation='merge')
    logging.info(f"Relationship created between {calendar_year.calendar_year_id} and {term_node.term_id}")

# Add each day to the correct term
for term_node in term_nodes:
    logging.info(f"Finding days for {term_node.term_id}...")
    term_start_date = term_node.term_start_date
    term_end_date = term_node.term_end_date
    logging.info(f"Term start date: {term_start_date}")
    logging.info(f"Term end date: {term_end_date}")
    for day in days:
        logging.debug(f"Checking if {day.date} is between {term_start_date} and {term_end_date}...")
        if term_start_date <= day.date <= term_end_date:
            logging.debug(f"Day {day.date} is between {term_start_date} and {term_end_date}")
            logging.debug(f"Creating relationship between {term_node.term_id} and {day.date}...")
            term_has_day = TermHasDay(source=term_node, target=day)
            neon.create_or_merge_neontology_relationship(term_has_day, operation='merge')
            logging.debug(f"Relationship created between {term_node.term_id} and {day.date}")
    
    # Add each week to the correct term
    for week in weeks:
        week_start_date = week.week_start

        logging.debug(f"Checking if {week_start_date} is between {term_start_date} and {term_end_date}...")
        if term_start_date <= week_start_date <= term_end_date:
            logging.debug(f"Week {week_start_date} is between {term_start_date} and {term_end_date}")
            logging.debug(f"Creating relationship between {term_node.term_id} and {week_start_date}...")
            term_has_week = TermHasWeek(source=term_node, target=week)
            neon.create_or_merge_neontology_relationship(term_has_week, operation='merge')
            logging.debug(f"Relationship created between {term_node.term_id} and {week_start_date}")
            

In [ ]:
# Extracting the number of teaching periods and times for each period
teaching_periods_count = calendar_lookup_df.loc[calendar_lookup_df['Identifier'] == 'TeachingPeriods', 'Data'].values[0]

# Creating a structure to hold period times
period_times = {}

for period_num in range(1, teaching_periods_count + 1):
    logging.info(f"Extracting period times for period {period_num}...")
    start_time_identifier = f'P.{period_num}.PeriodStartTime'
    end_time_identifier = f'P.{period_num}.PeriodEndTime'
    logging.debug(f"Start time identifier: {start_time_identifier}")
    logging.debug(f"End time identifier: {end_time_identifier}")

    # Extracting and converting the start and end times
    period_start_time = calendar_lookup_df.loc[calendar_lookup_df['Identifier'] == start_time_identifier, 'Data'].values[0]
    period_end_time = calendar_lookup_df.loc[calendar_lookup_df['Identifier'] == end_time_identifier, 'Data'].values[0]
    logging.debug(f"Period start time: {period_start_time}")
    logging.debug(f"Period end time: {period_end_time}")
    period_start_time = convert_time_string(period_start_time)
    period_end_time = convert_time_string(period_end_time)
    logging.pedantic(f"Period start time: {period_start_time}")
    logging.pedantic(f"Period end time: {period_end_time}")

    # Adding the period times to the structure
    period_times[f"P{period_num}"] = {
        "start_time": period_start_time,
        "end_time": period_end_time
    }
    logging.debug(f"Period times added to structure: {period_times}")

logging.info(f"Period times: {period_times}")


In [ ]:
# Retrieve the Calendar Year Node
calendar_year_node = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'CalendarYear', {'calendar_year_id': '2023-2024'})[0]
calendar_year_properties = dict(calendar_year_node._properties)

# Extract start and end dates
calendar_year_start_date = date(calendar_year_properties['calendar_year_start_date'].year, calendar_year_properties['calendar_year_start_date'].month, calendar_year_properties['calendar_year_start_date'].day)
calendar_year_end_date = date(calendar_year_properties['calendar_year_end_date'].year, calendar_year_properties['calendar_year_end_date'].month, calendar_year_properties['calendar_year_end_date'].day)

n = 0

# Process Day Nodes within the Calendar Year
day_nodes = neo4j.find_nodes_by_label(neo4j_driver, 'Day')
for day_node_object in day_nodes:
    day_node_properties = dict(day_node_object._properties)
    day_node = DayNode(**day_node_properties)
    
    # Convert day_node.date to a date object if it's not already
    day_node_date = pd.to_datetime(day_node.date).date()
    
    # Check if the day is within the calendar year and is a school day
    if calendar_year_start_date <= day_node_date <= calendar_year_end_date and day_node.day_modifier == 'School day':
        for period_num in range(1, teaching_periods_count + 1):
            # Create PeriodNode instance
            period_node = PeriodNode(
                period_id=f"{day_node.day_id}_P{period_num}",
                period_name=f"Period {period_num}",
                period_start_time=period_times[f"P{period_num}"]['start_time'],
                period_end_time=period_times[f"P{period_num}"]['end_time'],
                period_is_type='Academic',
                period_mod='',
                period_notes=f"Period {period_num} on {day_node.date}"
            )
            # Merge the PeriodNode instance and create relationships
            neon.create_or_merge_neontology_node(period_node, operation='merge')
            day_has_period = DayHasPeriod(source=day_node, target=period_node)
            neon.create_or_merge_neontology_relationship(day_has_period, operation='merge')
